In [8]:
from swiplserver import PrologMQI, PrologThread
import MetaTrader5 as mt5  # pip install MetaTrader5
import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly
from datetime import datetime, timedelta
import csv
import os
import threading
import time

# credentials
path = 'C:\\Program Files\\\MetaTrader 5\\terminal64.exe'
account = 6177262
password = 'torsn4qm'
server = 'OANDA-Demo-1'


In [9]:
# Function to start Meta Trader 5 (MT5)
def start_mt5(username, password, server, path):
    uname = int(username)
    pword = str(password)
    trading_server = str(server)
    filepath = str(path)

    # Attempt to start MT5
    if mt5.initialize(login=uname, password=pword, server=trading_server, path=filepath):
        print("Trading Bot Starting")
        # Login to MT5
        if mt5.login(login=uname, password=pword, server=trading_server):
            print("Trading Bot Logged in and Ready to Trade!")

            account_info = mt5.account_info()
            print(account_info)

            # getting specific account data
            account_name = account_info.name
            login_number = account_info.login
            balance = account_info.balance
            equity = account_info.equity

            print()
            print('name: ', account_name)
            print('login: ', login_number)
            print('balance: ', balance)
            print('equity: ', equity)
            return True
        else:
            print("Login Fail")
            quit()
            return PermissionError
    else:
        print("MT5 Initialization Failed")
        quit()
        return ConnectionAbortedError

def open_trade(action):
    # prepare the buy request structure
    symbol = 'EURUSD'

    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action == 'sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid


    buy_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": "EURUSD",
        "volume": 0.1,  # FLOAT
        "type": trade_type,
        "price": price,
        "sl": 0.0,  # FLOAT
        "tp": 0.0,  # FLOAT
        "deviation": 20,  # INTERGER
        "magic": 234000,  # INTERGER
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a trading request
    result = mt5.order_send(buy_request)
    return result, buy_request


def close_trade(ticket, action):
    """https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py"""
    # create a close request
    symbol = 'EURUSD'
    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action == 'sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid

    close_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": "EURUSD",
        "volume": 0.1,
        "type": trade_type,
        "position": ticket,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC,  # good till cancelled
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a close request
    result = mt5.order_send(close_request)
    

def get_candlestick(timeframe):
    # Request the last 60 candlesticks for EURUSD M1

    symbol = "EURUSD"
    timeframe_value = timeframe
    # Calculate the start and end time for the last week
    end_time = datetime.now()
    if timeframe == "h4":
        timeframe_time = mt5.TIMEFRAME_H4
        #start_time = end_time - timedelta(days=15)
    elif timeframe == "h1":
        timeframe_time = mt5.TIMEFRAME_H1
        #start_time = end_time - timedelta(days=5)
    elif timeframe == "m15":
        timeframe_time = mt5.TIMEFRAME_M15
        #start_time = end_time - timedelta(days=3)

    rates = mt5.copy_rates_range(symbol, timeframe_time, datetime(
        2023, 5, 23), datetime(2023, 6, 30))

    # total_candles = 60
    # candlesticks = mt5.copy_rates_from_pos(symbol, timeframe_m1, 0, total_candles)

    # Convert the candlestick data to a pandas DataFrame
    df = pd.DataFrame(rates)

    # Convert the timestamp to a human-readable format
    df['time'] = pd.to_datetime(df['time'], unit='s')

    fig = px.line(rates, x=df['time'], y=df['close'])
    fig.show()

    # Rearrange the columns to match the OHLC format
    ohlc_data = df[['time', 'open', 'high', 'low', 'close']]

    ohlc_data_to_display = df[['time', 'open', 'high', 'low', 'close']]

    ohlc_data_to_display.to_csv(
        'ohlc_data_with_time.csv', index=True, header=True)

    ohlc_data.to_csv('ohlc_data_full.csv', index=False, header=False)

    print(ohlc_data_to_display)

    return timeframe_value


In [10]:
class PrologThread(threading.Thread):
    def __init__(self, prolog_mqi):
        super().__init__()
        self.prolog_mqi = prolog_mqi
        self.expert_result_order = None  # Initialize expert_result attribute
        self.expert_result_time = None

    def run(self):
        with self.prolog_mqi as mqi:
            with mqi.create_thread() as prolog_expert_system:
                # Consult the file
                prolog_expert_system.query("consult('main_es.pl')")

                # Query the main/0 predicate
                prolog_expert_system.query("main.")
                result_order = prolog_expert_system.query("fact(order(X))")
                result_time = prolog_expert_system.query("fact(close_value(_, Time))")
                self.expert_result_order = result_order[0]['X']     # Store result_order
                self.expert_result_time = result_time[0]['Time']    # Store result_time
                

    def get_expert_result(self):
        return self.expert_result_order, self.expert_result_time
    

def expert_system():
    # Create an instance of PrologMQI
    prolog_mqi = PrologMQI()

    # Create an instance of PrologThread and pass the prolog_mqi instance as an argument
    prolog_thread = PrologThread(prolog_mqi)

    # Start the thread
    prolog_thread.start()

    # Wait for the thread to finish
    if prolog_thread.join():
        time.sleep(1)
    
    # Retrieve the expert_result using the get_expert_result() method
    expert_result_order, expert_result_time = prolog_thread.get_expert_result()
    
    return expert_result_order, expert_result_time


In [11]:
def extract_rows(timeframe):
    input_file = 'ohlc_data_full.csv'
    output_file = 'ohlc_data.csv'
    
    if timeframe == 'h4':
        num_rows = 50
    elif timeframe == 'h1':
        num_rows = 200
    elif timeframe == 'm15':
        num_rows = 800
    
    
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        # Read the specified number of rows
        rows = [row for row in reader][:num_rows]

    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)  # Write the selected rows

    # Update the input file by removing the extracted rows
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        # Read the remaining rows, skipping the extracted rows
        remaining_rows = list(reader)[num_rows:]

    with open(input_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(remaining_rows)  # Write the remaining rows


def extract_and_update_data(timeframe):
    input_file_full = 'ohlc_data_full.csv'
    input_file = 'ohlc_data.csv'
    
    if timeframe == 'h4':
        num_rows = 6
    elif timeframe == 'h1':
        num_rows = 24
    elif timeframe == 'm15':
        num_rows = 96

    # Check if the input file is empty
    if os.path.getsize(input_file_full) == 0:
        print(f"{input_file_full} is empty. Stopping criterion met.")
        return False

    # Extract the first six rows from ohlc_data_full.csv
    extracted_rows = []
    with open(input_file_full, 'r') as file:
        reader = csv.reader(file)
        extracted_rows = [row for row in reader][:num_rows]

    # Read the remaining rows from ohlc_data.csv
    remaining_rows = []
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        remaining_rows = list(reader)[num_rows:]

    # Update ohlc_data.csv by shifting remaining rows and inserting extracted rows at the bottom
    with open(input_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(remaining_rows + extracted_rows)

    # Remove the first six rows from ohlc_data_full.csv
    with open(input_file_full, 'r') as file:
        reader = csv.reader(file)
        remaining_rows_full = list(reader)[num_rows:]

    with open(input_file_full, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(remaining_rows_full)

    return True


MAIN FUNCTION:

In [12]:
#   Start the MT5
start_mt5(account, password, server, path)

# total number of positions
num_positions = mt5.positions_total()

print(num_positions)

# list of positions
positions = mt5.positions_get()

print(positions)


Trading Bot Starting
Trading Bot Logged in and Ready to Trade!
AccountInfo(login=6177262, trade_mode=0, leverage=100, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=99936.4, credit=0.0, profit=0.0, equity=99936.4, margin=0.0, margin_free=99936.4, margin_level=0.0, margin_so_call=100.0, margin_so_so=50.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jaafar Omar', server='OANDA-Demo-1', currency='USD', company='OANDA Global Markets')

name:  Jaafar Omar
login:  6177262
balance:  99936.4
equity:  99936.4
0
()


In [13]:
# Get Candlestick
timeframe = get_candlestick("h4")
timeframe


                   time     open     high      low    close
0   2023-05-22 16:00:00  1.08190  1.08241  1.07958  1.08094
1   2023-05-22 20:00:00  1.08094  1.08212  1.08077  1.08124
2   2023-05-23 00:00:00  1.08096  1.08203  1.08059  1.08114
3   2023-05-23 04:00:00  1.08113  1.08116  1.08022  1.08099
4   2023-05-23 08:00:00  1.08099  1.08126  1.07847  1.08029
..                  ...      ...      ...      ...      ...
164 2023-06-29 00:00:00  1.09113  1.09185  1.09008  1.09067
165 2023-06-29 04:00:00  1.09067  1.09143  1.08803  1.08850
166 2023-06-29 08:00:00  1.08850  1.09215  1.08838  1.09140
167 2023-06-29 12:00:00  1.09144  1.09410  1.08728  1.08807
168 2023-06-29 16:00:00  1.08808  1.08963  1.08598  1.08704

[169 rows x 5 columns]


'h4'

In [14]:
extract_rows(timeframe)

In [15]:
sum_sell = 0
sum_buy = 0
sum_hold = 0

In [16]:
while extract_and_update_data(timeframe):

    resultOrder, resultTime = expert_system()

    if resultOrder == 'sell':
        sum_sell = sum_sell + 1
    elif resultOrder == 'buy':
        sum_buy = sum_buy + 1
    elif resultOrder == 'hold':
        sum_hold = sum_hold + 1
    else:
        sum_hold = sum_hold + 1

Prolog: Date and Time: 2023-06-02 20:00:00
Prolog: Rule fired 18
Prolog: Order: hold
Prolog: 
Prolog: Date and Time: 2023-06-05 20:00:00
Prolog: Rule fired 9
Prolog: Order: hold
Prolog: 
Prolog: Date and Time: 2023-06-06 20:00:00
Prolog: Rule fired 18
Prolog: Order: hold
Prolog: 
Prolog: Date and Time: 2023-06-07 20:00:00
Prolog: Rule fired 20
Prolog: Order: hold
Prolog: 
Prolog: Date and Time: 2023-06-08 20:00:00
Prolog: Rule fired 3
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-09 20:00:00
Prolog: Rule fired 1
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-12 20:00:00
Prolog: Rule fired 2
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-13 20:00:00
Prolog: Rule fired 3
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-14 20:00:00
Prolog: Rule fired 3
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-15 20:00:00
Prolog: Rule fired 2
Prolog: Order: buy
Prolog: 
Prolog: Date and Time: 2023-06-16 20:00:00
Prolog: Rule fired 2
Prolog: O

ohlc_data_full.csv is empty. Stopping criterion met.


In [17]:
print("Sell: ", sum_sell)
print("Buy: ", sum_buy)
print("Hold: ", sum_hold)


Sell:  1
Buy:  11
Hold:  8


In [18]:
total_sum = sum_sell + sum_buy + sum_hold

sell_percentage = (sum_sell / total_sum) * 100
buy_percentage = (sum_buy / total_sum) * 100
hold_percentage = (sum_hold / total_sum) * 100

print("Sell: ", sell_percentage, "%")
print("Buy: ", buy_percentage, "%")
print("Hold: ", hold_percentage, "%")

Sell:  5.0 %
Buy:  55.00000000000001 %
Hold:  40.0 %
